In [4]:
import numpy as np
import pandas as pd

# Constants from the technical specification
NUM_AGENTS = 100
STEPS = 1000
COLLISION_THRESHOLD = 5.0
VELOCITY_DAMPING = 0.9
MIN_DISTANCE = 3.0
AGENT_MASS = 1.0
g = 9.81
FIXED_AVG_SPEED = 0.6
OPTIMIZATION_SEPARATION_RADIUS = 8.0

PARAMETERS = {
    "vicsek": {"noise": 0.04, "alignment": 0.08, "cohesion": 0.005, "perception_radius": 10.0},
    "boids": {"cohesion": 0.006, "separation": 0.2, "alignment": 0.08, "perception_radius": 10.0},
    "pso": {"global_attraction": 0.06, "personal_attraction": 0.04, "inertia": 0.8, "perception_radius": 12.0},
    "potential": {"repulsion": 0.08, "attraction": 0.04, "range": 14.0, "decay_factor": 1.5},
    "optimization": {"global_attraction": 0.04, "separation": 0.6, "formation": 0.12, "perception_radius": 9.0}
}

class SwarmSimulation:
    def __init__(self, num_agents, algorithm):
        self.num_agents = num_agents
        self.algorithm = algorithm
        self.positions = np.random.rand(num_agents, 2) * 50
        self.velocities = (np.random.rand(num_agents, 2) * 2 - 1)
        self.limit_speed()
        self.local_best = np.copy(self.positions)
        self.global_best = np.mean(self.positions, axis=0)
        self.relative_distances = np.zeros((num_agents, num_agents))
        self.relative_angles = np.zeros((num_agents, num_agents))
        self.update_local_information()

    def update_local_information(self):
        for i in range(self.num_agents):
            for j in range(self.num_agents):
                self.relative_distances[i, j] = np.linalg.norm(self.positions[i] - self.positions[j])
        for i in range(self.num_agents):
            for j in range(self.num_agents):
                if i != j:
                    relative_position = self.positions[j] - self.positions[i]
                    angle_i = np.arctan2(self.velocities[i][1], self.velocities[i][0])
                    angle_j = np.arctan2(relative_position[1], relative_position[0])
                    self.relative_angles[i, j] = (angle_j - angle_i) % (2 * np.pi)

    def limit_speed(self):
        speeds = np.linalg.norm(self.velocities, axis=1)
        avg_speed = np.mean(speeds)
        if avg_speed > 0:
            self.velocities *= FIXED_AVG_SPEED / avg_speed

    def normalize_velocities(self):
        for i in range(self.num_agents):
            speed = np.linalg.norm(self.velocities[i])
            if speed > 0:
                self.velocities[i] = self.velocities[i] / speed * FIXED_AVG_SPEED

    def vicsek_step(self):
        params = PARAMETERS["vicsek"]
        perception_radius = params["perception_radius"]
        new_velocities = np.zeros_like(self.velocities)
        for i in range(self.num_agents):
            neighbors = np.where(self.relative_distances[i] < perception_radius)[0]
            neighbors = neighbors[neighbors != i]
            if len(neighbors) > 0:
                avg_direction = np.zeros(2)
                for j in neighbors:
                    angle_i = np.arctan2(self.velocities[i][1], self.velocities[i][0])
                    abs_angle = (angle_i + self.relative_angles[i, j]) % (2 * np.pi)
                    neighbor_dir = np.array([np.cos(abs_angle), np.sin(abs_angle)])
                    avg_direction += neighbor_dir
                if np.linalg.norm(avg_direction) > 0:
                    avg_direction /= np.linalg.norm(avg_direction)
                center_of_mass = np.zeros(2)
                for j in neighbors:
                    rel_position = np.array([
                        self.relative_distances[i, j] * np.cos(self.relative_angles[i, j]),
                        self.relative_distances[i, j] * np.sin(self.relative_angles[i, j])
                    ])
                    center_of_mass += rel_position
                center_of_mass /= len(neighbors)
                cohesion_dir = center_of_mass / (np.linalg.norm(center_of_mass) + 1e-8)
                alignment = avg_direction * params["alignment"]
                cohesion = cohesion_dir * params["cohesion"]
                noise = (np.random.rand(2) - 0.5) * params["noise"]
                new_velocities[i] = self.velocities[i] + alignment + cohesion + noise
            else:
                new_velocities[i] = self.velocities[i]
        self.velocities = new_velocities
        self.normalize_velocities()

    def boids_step(self):
        params = PARAMETERS["boids"]
        perception_radius = params["perception_radius"]
        new_velocities = np.zeros_like(self.velocities)
        for i in range(self.num_agents):
            neighbors = np.where(self.relative_distances[i] < perception_radius)[0]
            neighbors = neighbors[neighbors != i]
            alignment = cohesion = separation = np.zeros(2)
            if len(neighbors) > 0:
                avg_velocity = np.zeros(2)
                for j in neighbors:
                    angle_i = np.arctan2(self.velocities[i][1], self.velocities[i][0])
                    abs_angle = (angle_i + self.relative_angles[i, j]) % (2 * np.pi)
                    speed_j = np.linalg.norm(self.velocities[j])
                    neighbor_vel = speed_j * np.array([np.cos(abs_angle), np.sin(abs_angle)])
                    avg_velocity += neighbor_vel
                avg_velocity /= len(neighbors)
                alignment = (avg_velocity - self.velocities[i]) * params["alignment"]
                center_of_mass = np.zeros(2)
                for j in neighbors:
                    rel_position = np.array([
                        self.relative_distances[i, j] * np.cos(self.relative_angles[i, j]),
                        self.relative_distances[i, j] * np.sin(self.relative_angles[i, j])
                    ])
                    center_of_mass += rel_position
                center_of_mass /= len(neighbors)
                cohesion = center_of_mass * params["cohesion"]
                for j in neighbors:
                    if self.relative_distances[i, j] < MIN_DISTANCE:
                        rel_position = np.array([
                            self.relative_distances[i, j] * np.cos(self.relative_angles[i, j]),
                            self.relative_distances[i, j] * np.sin(self.relative_angles[i, j])
                        ])
                        repulsion = -rel_position / (np.linalg.norm(rel_position)**2 + 1e-8)
                        separation += repulsion
                separation *= params["separation"]
            new_velocities[i] = self.velocities[i] + alignment + cohesion + separation
        self.velocities = new_velocities
        self.normalize_velocities()

    def pso_step(self):
        params = PARAMETERS["pso"]
        target_distances = np.array([self.evaluate_position(i) for i in range(self.num_agents)])
        for i in range(self.num_agents):
            if self.evaluate_position(i) < self.evaluate_local_best(i):
                self.local_best[i] = self.positions[i].copy()
        best_idx = np.argmin(target_distances)
        global_best = self.positions[best_idx].copy()
        for i in range(self.num_agents):
            inertia = self.velocities[i] * params["inertia"]
            personal_attraction_direction = self.local_best[i] - self.positions[i]
            personal_attraction = personal_attraction_direction * params["personal_attraction"] * np.random.rand()
            global_attraction_direction = global_best - self.positions[i]
            global_attraction = global_attraction_direction * params["global_attraction"] * np.random.rand()
            self.velocities[i] = inertia + personal_attraction + global_attraction
        self.normalize_velocities()

    def evaluate_position(self, agent_idx):
        return np.linalg.norm(self.positions[agent_idx] - self.global_best)

    def evaluate_local_best(self, agent_idx):
        return np.linalg.norm(self.local_best[agent_idx] - self.global_best)

    def potential_field_step(self):
        params = PARAMETERS["potential"]
        new_velocities = np.zeros_like(self.velocities)
        for i in range(self.num_agents):
            repulsion = np.zeros(2)
            for j in range(self.num_agents):
                if i != j and self.relative_distances[i, j] < params["range"]:
                    decay = np.exp(-self.relative_distances[i, j] / params["decay_factor"])
                    repulsion_force = params["repulsion"] * decay
                    angle = self.relative_angles[i, j]
                    repulsion += -repulsion_force * np.array([np.cos(angle), np.sin(angle)])
            angle_bins = np.linspace(0, 2*np.pi, 8, endpoint=False)
            density = np.zeros(len(angle_bins))
            for k, angle in enumerate(angle_bins):
                density_in_direction = 0
                for j in range(self.num_agents):
                    if i != j:
                        angle_diff = abs((self.relative_angles[i, j] - angle) % (2*np.pi))
                        if angle_diff < np.pi/4:
                            density_in_direction += 1 / (self.relative_distances[i, j] + 1e-8)
                density[k] = density_in_direction
            least_dense_direction = angle_bins[np.argmin(density)]
            attraction = params["attraction"] * np.array([np.cos(least_dense_direction), np.sin(least_dense_direction)])
            new_velocities[i] = self.velocities[i] + repulsion + attraction
        self.velocities = new_velocities
        self.normalize_velocities()

    def optimization_step(self):
        params = PARAMETERS["optimization"]
        for i in range(self.num_agents):
            avg_direction = np.zeros(2)
            for j in range(self.num_agents):
                if i != j:
                    angle_i = np.arctan2(self.velocities[i][1], self.velocities[i][0])
                    abs_angle_j = (angle_i + self.relative_angles[i, j]) % (2 * np.pi)
                    direction_j = np.array([np.cos(abs_angle_j), np.sin(abs_angle_j)])
                    avg_direction += direction_j
            if np.linalg.norm(avg_direction) > 0:
                avg_direction /= np.linalg.norm(avg_direction)
            formation_attraction = avg_direction * params["formation"]

            separation = np.zeros(2)
            for j in range(self.num_agents):
                if i != j and self.relative_distances[i, j] < OPTIMIZATION_SEPARATION_RADIUS:
                    angle = self.relative_angles[i, j]
                    repulsion = -np.array([np.cos(angle), np.sin(angle)])
                    separation += repulsion / (self.relative_distances[i, j]**2 + 1e-8)

            separation *= params["separation"]  # Removed multiplier 0.5
            global_attraction = avg_direction * params["global_attraction"]
            self.velocities[i] += formation_attraction + separation + global_attraction

        self.normalize_velocities()

    def update(self):
        if self.algorithm == "vicsek":
            self.vicsek_step()
        elif self.algorithm == "boids":
            self.boids_step()
        elif self.algorithm == "pso":
            self.pso_step()
        elif self.algorithm == "potential":
            self.potential_field_step()
        elif self.algorithm == "optimization":
            self.optimization_step()
        self.limit_speed()
        self.positions += self.velocities
        self.positions[:, 1] = np.clip(self.positions[:, 1], 0, None)
        #self.positions = np.clip(self.positions, 0, 50)
        self.update_local_information()

    def compute_metrics(self):
        distances = np.linalg.norm(self.positions[:, np.newaxis] - self.positions, axis=2)
        avg_dist = np.mean(distances)
        max_dist = np.max(distances)
        target_deviation = np.mean(np.linalg.norm(self.positions - self.global_best, axis=1))
        collisions = np.sum((distances > 0) & (distances < COLLISION_THRESHOLD)) // 2
        avg_speed = np.mean(np.linalg.norm(self.velocities, axis=1))
        stability = 1 / (1 + target_deviation / 10)
        efficiency = avg_speed / (1 + avg_dist / 10)
        kinetic_energy = 0.5 * AGENT_MASS * np.linalg.norm(self.velocities, axis=1) ** 2
        avg_kinetic_energy = np.mean(kinetic_energy)
        max_kinetic_energy = np.max(kinetic_energy)
        potential_energy = AGENT_MASS * g * self.positions[:, 1]
        avg_potential_energy = np.mean(potential_energy)
        max_potential_energy = np.max(potential_energy)
        return {
            "Average distance": avg_dist,
            "Maximum distance": max_dist,
            "Average deviation": target_deviation,
            "Number of collisions": collisions,
            "Average speed": avg_speed,
            "Formation stability": stability,
            "Energy efficiency": efficiency,
            "Average kinetic energy": avg_kinetic_energy,
            "Maximum kinetic energy": max_kinetic_energy,
            "Average potential energy": avg_potential_energy,
            "Maximum potential energy": max_potential_energy
        }

    def simulate(self, steps):
        for _ in range(steps):
            self.update()
        return self.compute_metrics()

def categorize_metrics(results_df, thresholds):
    categorized = pd.DataFrame(index=results_df.index, columns=results_df.columns)
    for metric in thresholds.keys():
        for alg in results_df.index:
            value = results_df.loc[alg, metric]
            cats = thresholds[metric]
            if value < cats["Average"]: category = "Low"
            elif value < cats["High"]: category = "Average"
            else: category = "High"
            categorized.loc[alg, metric] = category
    return categorized

# Main code
algorithms = ["vicsek", "boids", "pso", "potential", "optimization"]
results = {alg: SwarmSimulation(NUM_AGENTS, alg).simulate(STEPS) for alg in algorithms}
results_df = pd.DataFrame(results).T

thresholds = {
    "Average distance": {"Low": 10, "Average": 20, "High": 30},
    "Maximum distance": {"Low": 30, "Average": 50, "High": 70},
    "Average deviation": {"Low": 15, "Average": 30, "High": 45},
    "Number of collisions": {"Low": 200, "Average": 500, "High": 1000},
    "Average speed": {"Low": 0.3, "Average": 0.6, "High": 1.0},
    "Formation stability": {"Low": 0.3, "Average": 0.6, "High": 0.9},
    "Energy efficiency": {"Low": 0.1, "Average": 0.2, "High": 0.3}
}

print("Numerical simulation results:")
print(results_df.round(2))
print("\nCategorized results:")
print(categorize_metrics(results_df[list(thresholds.keys())], thresholds))

theoretical_df = pd.DataFrame({
    "vicsek": {"Average distance": "Average", "Maximum distance": "High", "Average deviation": "High", "Number of collisions": "Low", "Average speed": "Average", "Formation stability": "High", "Energy efficiency": "Average"},
    "boids": {"Average distance": "Average", "Maximum distance": "Average", "Average deviation": "Average", "Number of collisions": "Low", "Average speed": "Average", "Formation stability": "High", "Energy efficiency": "High"},
    "pso": {"Average distance": "High", "Maximum distance": "Average", "Average deviation": "Low", "Number of collisions": "Low", "Average speed": "High", "Formation stability": "High", "Energy efficiency": "High"},
    "potential": {"Average distance": "High", "Maximum distance": "Average", "Average deviation": "Low", "Number of collisions": "Low", "Average speed": "High", "Formation stability": "High", "Energy efficiency": "High"},
    "optimization": {"Average distance": "High", "Maximum distance": "Average", "Average deviation": "Low", "Number of collisions": "Low", "Average speed": "High", "Formation stability": "High", "Energy efficiency": "High"}}).T

print("\nTheoretical qualitative indicators:")
print(theoretical_df)

match_count = 0
total_count = 0
for alg in algorithms:
    for metric in theoretical_df.columns:
        total_count += 1
        if categorize_metrics(results_df, thresholds).loc[alg, metric] == theoretical_df.loc[alg, metric]:
            match_count += 1
print(f"\nCompliance with theoretical indicators: {match_count}/{total_count} ({match_count/total_count*100:.1f}%)")

Numerical simulation results:
              Average distance  Maximum distance  Average deviation  \
vicsek                  238.70           1059.27             170.30   
boids                   403.86           1233.66             402.69   
pso                       3.48              9.16               2.63   
potential               125.52            296.42             480.99   
optimization             14.04             43.01             223.10   

              Number of collisions  Average speed  Formation stability  \
vicsek                       141.0            0.6                 0.06   
boids                          2.0            0.6                 0.02   
pso                         3960.0            0.6                 0.79   
potential                     84.0            0.6                 0.02   
optimization                 438.0            0.6                 0.04   

              Energy efficiency  Average kinetic energy  \
vicsek                     0.02        